In [1]:
from fastai.vision.all import *
import fastai
import timm
import pandas as pd
from fastai.metrics import mae,rmse,mse

In [2]:
import torch
torch.cuda.set_device(0)

In [3]:
df = pd.read_csv('../dataset.csv')

In [4]:
df = df.drop(df[df.masa<0].index)

In [5]:
dftrain = df[(df.split=='train') | (df.split=='valid')]
dftest = df[(df.split=='test')]

In [6]:
def get_masa(f):
    return float(df[df['image_id']=='datasetClean/'+f.name]['masa'])

def get_volumen(f):
    return float(df[df['image_id']=='datasetClean/'+f.name]['volumen'])

def is_valid(f):
    return (df[df['image_id']=='datasetClean/'+f.name]['split']).values[0]=='valid'

In [7]:
def is_test(f):
    return (df[df['image_id']=='datasetClean/'+f.name]['split']).values[0]=='test'

In [8]:
dbtest = DataBlock(blocks = (ImageBlock, RegressionBlock()),
                 get_items=get_image_files, 
                 splitter=FuncSplitter(is_test),
                 get_y=get_masa,
                 item_tfms = Resize(224),
                 batch_tfms=[*aug_transforms(size=224,flip_vert=True,
                                            max_zoom=1.0), Normalize.from_stats(*imagenet_stats)])

In [9]:
path = Path('../datasetAltura2/')
trainPath = path/'train'

In [10]:
dlstest = dbtest.dataloaders(path,bs=8)

In [11]:
learn1 = Learner(dlstest, timm.create_model('convnext_base',num_classes=1,pretrained=True),
                loss_func=MSELossFlat(), metrics=[mae,rmse,mse]).to_fp16()

In [12]:
learn1.load('convnext_base_Masa_Height')

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [14]:
learn1.export('convnext_base_Masa_Height.pkl')

In [13]:
preds1 = learn1.get_preds()

In [14]:
preds1TTA = learn1.tta()

In [15]:
learn2 = Learner(dlstest, timm.create_model('efficientnetv2_rw_s',num_classes=1,pretrained=True),
                loss_func=MSELossFlat(), metrics=[mae,rmse,mse]).to_fp16()

In [16]:
learn2.load('efficientnetv2_rw_s_masa_height')

In [17]:
preds2 = learn2.get_preds()

In [18]:
preds2TTA = learn2.tta()

In [15]:
learn3 = Learner(dlstest, timm.create_model('resnetrs50',num_classes=1,pretrained=True),
                loss_func=MSELossFlat(), metrics=[mae,rmse,mse]).to_fp16()

In [16]:
learn3.load('resnetrs50Masav2Height')

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [17]:
learn3.export('resnetrs50Masav2Height.pkl')

In [21]:
preds3 = learn3.get_preds()

In [22]:
preds3TTA = learn3.tta()

In [23]:
learn4 = cnn_learner(dlstest,resnet50,loss_func=MSELossFlat(), metrics=[mae,rmse,mse]).to_fp16()

In [24]:
learn4.load('resnet50Masav2Height')

In [25]:
preds4 = learn4.get_preds()

In [26]:
preds4TTA = learn4.tta()

In [18]:
dbtest = DataBlock(blocks = (ImageBlock, RegressionBlock()),
                 get_items=get_image_files, 
                 splitter=FuncSplitter(is_test),
                 get_y=get_masa,
                 item_tfms = Resize(512),
                 batch_tfms=[*aug_transforms(size=512,flip_vert=True,
                                            max_zoom=1.0), Normalize.from_stats(*imagenet_stats)])

In [19]:
path = Path('../datasetClean/')

In [20]:
dlstest = dbtest.dataloaders(path,bs=8)

In [21]:
learn5 = Learner(dlstest, timm.create_model('convnext_base',num_classes=1,pretrained=True),
                loss_func=MSELossFlat(), metrics=mae).to_fp16()

In [22]:
learn5.load('convnext_base_Masa')

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [23]:
learn5.export('convnext_base_Masa.pkl')

In [32]:
preds5 = learn5.get_preds()

In [33]:
preds5TTA = learn5.tta()

In [34]:
learn6 = Learner(dlstest, timm.create_model('efficientnetv2_rw_s',num_classes=1,pretrained=True),
                loss_func=MSELossFlat(), metrics=mae).to_fp16()

In [35]:
learn6.load('efficientnetv2_rw_s_masa')

In [36]:
preds6 = learn6.get_preds()

In [37]:
preds6TTA = learn6.tta()

In [38]:
learn7 = Learner(dlstest, timm.create_model('resnetrs50',num_classes=1,pretrained=True),
                loss_func=MSELossFlat(), metrics=mae).to_fp16()

In [39]:
learn7.load('resnetrs50Masav2')

In [40]:
preds7 = learn7.get_preds()

In [41]:
preds7TTA = learn7.tta()

In [42]:
learn8 = cnn_learner(dlstest,resnet50,loss_func=MSELossFlat(), metrics=[mae,rmse,mse]).to_fp16()

In [43]:
learn8.load('resnet50Masav2')

In [44]:
preds8 = learn2.get_preds()

In [45]:
preds8TTA = learn8.tta()

In [57]:
mae((preds5[0]+preds6[0]+preds7[0])/3,preds1[1])

TensorBase(28.7668)

In [59]:
mae((preds5TTA[0]+preds6TTA[0]+preds7TTA[0])/3,preds1[1])

TensorBase(28.3314)